In [ ]:
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

SELECT name 
FROM Facilities 
WHERE membercost > 0;


In [ ]:
/* Q2: How many facilities do not charge a fee to members? */

4 facilities

In [ ]:
/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

SELECT name
FROM Facilities
WHERE membercost > 0
AND membercost < (0.2 * monthlymaintenance);


In [ ]:
/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

SELECT facid
FROM Facilities
WHERE facid IN (0,5)

In [ ]:
/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

SELECT name
    CASE
        WHEN monthlymaintenance > 100 THEN 'expensive'
        ELSE 'cheap'
    END AS cost_category
FROM Facilities;


In [ ]:
/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

SELECT firstname, surname
FROM Members
WHERE joindate = (SELECT MAX(joindate) FROM Members);


In [ ]:
/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

select distinct mems.firstname || ' ' || mems.surname as member, facs.name as facility
	from 
		cd.members mems
		inner join cd.bookings bks
			on mems.memid = bks.memid
		inner join cd.facilities facs
			on bks.facid = facs.facid
	where
		facs.name in ('Tennis Court 2','Tennis Court 1')
order by member, facility 


In [ ]:
/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

select mems.firstname || ' ' || mems.surname as member, 
	facs.name as facility, 
	case 
		when mems.memid = 0 then
			bks.slots*facs.guestcost
		else
			bks.slots*facs.membercost
	end as cost
        from
                cd.members mems                
                inner join cd.bookings bks
                        on mems.memid = bks.memid
                inner join cd.facilities facs
                        on bks.facid = facs.facid
        where
		bks.starttime >= '2012-09-14' and 
		bks.starttime < '2012-09-15' and (
			(mems.memid = 0 and bks.slots*facs.guestcost > 30) or
			(mems.memid != 0 and bks.slots*facs.membercost > 30)
		)
order by cost desc;          


In [ ]:
/* Q9: This time, produce the same result as in Q8, but using a subquery. */

select distinct mems.firstname || ' ' ||  mems.surname as member,
	(select recs.firstname || ' ' || recs.surname as recommender 
		from cd.members recs 
		where recs.memid = mems.recommendedby
	)
	from 
		cd.members mems
order by member;  


In [ ]:
/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

SELECT * 
FROM (
SELECT Facilities.name AS facility, CONCAT( Members.firstname,  ' ', Members.surname ) AS name, 
    CASE 
        WHEN Bookings.memid =0
        THEN Facilities.guestcost * Bookings.slots
    ELSE Facilities.membercost * Bookings.slots
END AS cost
FROM Bookings
INNER JOIN Facilities ON Bookings.facid = Facilities.facid
AND Bookings.starttime LIKE  '2012-09-14%'
INNER JOIN Members ON Bookings.memid = Members.memid
)sub
WHERE sub.cost >30
ORDER BY sub.cost DESC;


In [ ]:
/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

SELECT 
    m1.surname AS member_surname,
    m1.firstname AS member_firstname,
    m2.surname AS recommender_surname,
    m2.firstname AS recommender_firstname
FROM Members
LEFT JOIN Members m2 ON m1.recommendedby = m2.memid
ORDER BY member_surname, member_firstname;


In [ ]:
/* Q12: Find the facilities with their usage by member, but not guests */

SELECT 
    f.facid, 
    f.name AS facility_name, 
    SUM(b.slots) AS total_usage
FROM Facilities f
JOIN Bookings b ON f.facid = b.facid
WHERE b.memid <> 0  -- Excludes guest bookings
GROUP BY f.facid, f.name
ORDER BY total_usage DESC;

/* Q13: Find the facilities usage by month, but not guests */

SELECT 
    EXTRACT(MONTH FROM b.starttime) AS month, 
    f.facid, 
    f.name AS facility_name, 
    SUM(b.slots) AS total_usage
FROM Bookings b
JOIN Facilities f ON b.facid = f.facid
WHERE b.memid <> 0  -- Excludes guest bookings
GROUP BY month, f.facid, f.name
ORDER BY month, f.facid;